## Introduction

This notebook will introduce some concepts of font and text handling in the [**Cairo**](http://cairographics.org/) 2D graphics library. Access to Cairo in Python will be done via the [**Qahirah**](https://github.com/ldo/qahirah) high-level binding.

The functions described here make up what is called Cairo’s “toy” font API. This means they do not provide advanced layout features like ligatures, auto-kerning and bidirectional or vertical text.

The following topics will be covered:
* [Loading Fonts](#Loading-Fonts)
* [Measuring Text](#Measuring-Text)
* [Horizontally Aligning Text](#Horizontally-Aligning-Text)
* [Vertically Aligning Text](#Vertically-Aligning-Text)
 * [On The Text Metrics](#Vertical-Alignment-On-The-Text-Metrics)
 * [On The Font Metrics](#Vertical-Alignment-On-The-Font-Metrics)
* [Characters And Glyphs](#Characters-And-Glyphs)
* [Font Size Subtleties](#Font-Size-Subtleties)
* [The Font Matrix](#The-Font-Matrix)
* [Fitting Text To A Given Width](#Fitting-Text-To-A-Given-Width)

Start by executing the following cell to set up some common definitions needed later.

In [ ]:
from ipywidgets import \
    widgets
from ipywidgets.widgets import \
    interact
from IPython.display import \
    display_png
import qahirah as qah
from qahirah import \
    CAIRO, \
    Colour, \
    Matrix, \
    Vector

pix = qah.ImageSurface.create \
  (
    format = CAIRO.FORMAT_RGB24,
    dimensions = (600, 200)
  )
ctx = None

def reset() :
    "(re)initializes the drawing context, wiping out any existing drawing."
    global ctx
    del ctx
    ctx = qah.Context.create(pix)
    (ctx
       .save()
       .set_source_colour(Colour.grey(.95))
       .paint()
       .restore()
    )
#end reset

def display() :
    "(re)displays what has been drawn."
    display_png(pix.to_png_bytes(), raw = True)
#end display

reset()

## Loading Fonts

Cairo has two main object types to do with managing fonts: [cairo_font_face_t](http://cairographics.org/manual/cairo-cairo-font-face-t.html) (represented in Qahirah by a `FontFace` object) and [cairo_scaled_font_t](http://cairographics.org/manual/cairo-cairo-scaled-font-t.html) (represented in Qahirah by a `ScaledFontFace`). A *font face* is what you get when you load a font into Cairo, while a *scaled font face* is part of Cairo’s font-caching machinery: it is what you get after specifying a font size and other rendering options for the font. If you are displaying text with a number of different, repeated, settings, then it is more efficient to pre-setup some scaled font face objects for these settings, and switch between them, rather than specifying the font face, font size etc every time.

The simplest (and crudest) way to specify a font in Cairo is with [cairo_select_font_face](http://cairographics.org/manual/cairo-text.html#cairo-select-font-face). The documentation says that the `family` string is a “simplified description”, citing generic examples like “`serif`”, “`sans-serif`” and so on. In fact, on Linux systems, this string is interpreted as a [Fontconfig](http://www.freedesktop.org/wiki/Software/fontconfig/) pattern. But note that the “slant” and “weight” pattern elements are overridden by the corresponding arguments to `select_font_face`.

In [ ]:
sample_text = "Lorem Ipsum"
    
@interact \
  (
    fontspec = widgets.Text(value = "Serif"),
    fontsize = (12, 72, 12),
    italic = widgets.Checkbox(),
    bold = widgets.Checkbox(),
  )
def select_font_demo(fontspec, fontsize, italic, bold) :
    reset()
    ctx.translate(pix.dimensions * Vector(0.1, 0.6))
    ctx.select_font_face \
      (
        family = fontspec,
        slant = (CAIRO.FONT_SLANT_NORMAL, CAIRO.FONT_SLANT_ITALIC)[italic],
        weight = (CAIRO.FONT_WEIGHT_NORMAL, CAIRO.FONT_WEIGHT_BOLD)[bold],
      )
    ctx.set_font_size(fontsize)
    ctx.move_to((0, 0))
    ctx.show_text(sample_text)
    display()
#end select_font_demo

Qahirah supports additional ways of loading fonts, using the various `create` methods of the `FontFace` class: if you have [python_freetype](https://github.com/ldo/python_freetype) installed, then you can make direct [FreeType](http://www.freetype.org/) calls to load and manipulate `freetype2.Face` objects which you then pass to `qahirah.FontFace.create_for_ft_face` to wrap in Cairo font face objects. Fonts can be specified via Fontconfig pattern strings, or by giving pathnames of font files.

## Measuring Text

Cairo provides two kinds of text metrics: *font extents* and *text extents*. These are explained in detail in the Cairo documentation for the [<tt>cairo_font_extents_t</tt>](http://cairographics.org/manual/cairo-cairo-scaled-font-t.html#cairo-font-extents-t)  and [<tt>cairo_text_extents_t</tt>](http://cairographics.org/manual/cairo-cairo-scaled-font-t.html#cairo-text-extents-t) structures. Broadly, the font extents give overall design metrics for the font, while the text extents are supposed to be precise metrics for a given piece of text.

Note that the font extents are defined by the font designer, and need not correspond to the metrics of any actual text. **Do not use font extents for dimensioning any actual clipping bounds**, as this could cause clipping of text. Use them only for calculating layout placements, as they are intended to produce pleasing results this way.

The following code illustrates the meanings of the font extents fields. The *baseline* corresponds to the $y$-coordinate of the current point when the text is being drawn, and the other values are used to draw parallel lines offset from this. Note that the ascent value has to be *subtracted* from the baseline $y$-coordinate. The height value is meant to be used to offset successive lines of text; here it is subtracted from the baseline, to show the baseline of what would be the line above.

In [ ]:
sample_font_name = "Liberation Serif"
sample_font = qah.FontFace.create_for_pattern(sample_font_name)
sample_font_size = 72
text_colour = Colour.grey(0)
line_colour = Colour.from_hsva((0.9, 0.5, 1, 0.5))
label_font = \
    (qah.Context.create_for_dummy()
         .select_font_face("serif", CAIRO.FONT_SLANT_NORMAL, CAIRO.FONT_WEIGHT_NORMAL)
         .set_font_size(12)
         .scaled_font
    )

def show_font_metrics() :
    reset()
    ctx.translate(pix.dimensions * Vector(0.05, 0.6))
    ctx.set_font_face(sample_font)
    ctx.set_font_size(sample_font_size)
    font_extents = ctx.font_extents
    print(font_extents) # debug
    line_length = ctx.text_extents(sample_text).x_advance * 1.05
    ctx.set_source_colour(text_colour)
    ctx.move_to((0, 0))
    ctx.show_text(sample_text)
    ctx.scaled_font = label_font
    for \
        y, label \
    in \
        (
            (0, "baseline"),
            (- font_extents.ascent, "- ascent"),
            (font_extents.descent, "descent"),
            (- font_extents.height, "- height"),
        ) \
    :
        ctx.set_source_colour(line_colour)
        ctx.move_to((0, y)).rel_line_to((line_length, 0))
        pos = ctx.current_point
        ctx.stroke()
        ctx.move_to(pos)
        ctx.set_source_colour(text_colour)
        ctx.show_text(label)
    #end for
    display()
#end show_font_metrics

show_font_metrics()

The text metrics give a completely different set of numbers. In the next illustration, the spot on the left illustrates the *origin* at which the text is being drawn, while the spot on the right illustrates the new origin for drawing subsequent text. The offset between the two is a vector defined by the <tt>x_advance</tt> and <tt>y_advance</tt> fields of the text extents.

The rest of the text extents fields define the bounding box for the text being drawn. The dimensions of this rectangle are given by the <tt>width</tt> and <tt>height</tt> fields, while its top-left corner is positioned at the coordinates given by the <tt>x_bearing</tt> and <tt>y_bearing</tt> fields, relative to the text origin.

For convenience, Qahirah lets you directly obtain the rectangle as *text_extents*`.bounds`, and the advance as the vector *text_extents*`.advance`.

In [ ]:
def show_text_metrics() :
    reset()
    ctx.translate(pix.dimensions * Vector(0.05, 0.6))
    ctx.set_font_face(sample_font)
    ctx.set_font_size(sample_font_size)
    text_extents = ctx.text_extents(sample_text)
    print(text_extents) # debug
    ctx.set_source_colour(text_colour)
    ctx.move_to((0, 0))
    ctx.show_text(sample_text)
    ctx.scaled_font = label_font
    ctx.set_source_colour(line_colour)
    ctx.new_path()
    ctx.rectangle(text_extents.bounds)
    ctx.stroke()
    for \
        pos \
    in \
        (
            Vector(0, 0),
            text_extents.advance,
        ) \
    :
        ctx.arc(pos, 4, 0, qah.circle, False)
        ctx.fill()
    #end for
    display()
#end show_text_metrics

show_text_metrics()

## Horizontally Aligning Text

There is often a need to draw text horizontally centred or right-aligned. The basic technique is very simple: use the <tt>x_advance</tt> field returned from the text metrics as the measure of the width of the text, and you can adjust the starting point for drawing the text appropriately.

The following routine demonstrates the effect of a range of possible alignments, controlled by an <tt>align</tt> parameter that varies continuously from 0 for left-aligned, through 0.5 for centred, to 1.0 for right-aligned.

In [ ]:
@interact(align = (0.0, 1.0, 0.05))
def text_alignment_demo(align) :
    reset()
    sample_text = \
        [
            "Lorem Ipsum Dolor",
            "Sit Amet",
        ]
    ctx.translate(pix.dimensions * Vector(0.5, 0.5))
    ctx.set_font_face(sample_font)
    ctx.set_font_size(36)
    align_point = Vector(0, 0 - ctx.font_extents.height * (len(sample_text) - 1) / 2)
    ctx.move_to((0, -100))
    ctx.rel_line_to((0, 200))
    ctx.source_colour = line_colour
    ctx.stroke()
    ctx.source_colour = text_colour
    for line in sample_text :
        text_width = ctx.text_extents(line).x_advance
        ctx.move_to(align_point - Vector(align * text_width, 0))
        ctx.show_text(line)
        align_point += Vector(0, ctx.font_extents.height)
    #end for
    display()
#end text_alignment_demo

**Exercise:** What happens if you allow values for <tt>align</tt> outside of the $[0, 1]$ interval?

## Vertically Aligning Text

There is also sometimes a need to align text vertically—for example, for drawing a label in an on-screen button. This can be done either based on the text metrics of the specific label text, or on the overall font metrics.

### Vertical Alignment On The Text Metrics

The idea here is to vertically centre the bounding box of the text about the desired $y$-coordinate. If you remember, the height of the bounding box is the `height` field of the `text_extents`, while the vertical offset from the text origin to the top of the bounding box is the `y_bearing` field (usually negative).

If we use $y_o$ for the $y$-coordinate of the text origin, $b_y$ for the $y$-bearing, $h$ for the height of the bounding box, $y_t$ for the top of the bounding box and $y_b$ for the bottom of the bounding box, then

$$y_t = y_o + b_y$$

and

$$y_b = y_t + h = y_o + b_y + h$$

from which the mid $y$-coordinate of the bounding box, $y_m$, is

$$y_m = {{y_t + y_b} \over 2} = {y_o + b_y + {h \over 2}}$$

Therefore, given the desired vertical midpoint $y_m$ for the bounding box of the text, the necessary $y$-origin at which to draw it is

$$y_o = {y_m - b_y - {h \over 2}}$$

In the following code, $y_m$ comes from `label_pos.y`.

In [ ]:
def vert_align_text_metrics() :
    reset()
    label_pos = pix.dimensions * Vector(0.1, 0.5)
    ctx.source_colour = line_colour
    ctx.move_to(label_pos + Vector(- 300, 0))
    ctx.rel_line_to(Vector(900, 0))
    ctx.stroke()
    ctx.source_colour = text_colour
    ctx.set_font_face(sample_font)
    ctx.set_font_size(72)
    text_extents = ctx.text_extents(sample_text)
    ctx.move_to(label_pos + Vector(0, - text_extents.y_bearing - text_extents.height / 2))
    ctx.show_text(sample_text)
    display()
#end vert_align_text_metrics

vert_align_text_metrics()

### Vertical Alignment On The Font Metrics

Similar to the above, this time the bounding box we wish to centre vertically is computed from the font metrics. Using the same symbols $y_t$, $y_b$, $y_m$ as before for the top, bottom and middle of the bounding box and $y_o$ for the text $y$-origin, this time adding $a$ for the font ascent and $d$ for the font descent (both positive), then

$$y_t = y_o - a$$
$$y_b = y_o + d$$

from which

$$y_m = {{y_t + y_b} \over 2} = y_o + {{d - a} \over 2}$$

Therefore, for a given $y_m$, the necessary text $y$-origin is

$$y_o = y_m + {{a - d} \over 2}$$

As before, $y_m$ in the following code comes from `label_pos.y`.

In [ ]:
def vert_align_font_metrics() :
    reset()
    label_pos = pix.dimensions * Vector(0.1, 0.5)
    ctx.source_colour = line_colour
    ctx.move_to(label_pos + Vector(- 300, 0))
    ctx.rel_line_to((900, 0))
    ctx.stroke()
    ctx.source_colour = text_colour
    ctx.set_font_face(sample_font)
    ctx.set_font_size(72)
    font_extents = ctx.font_extents
    ctx.move_to(label_pos + Vector(0, (font_extents.ascent - font_extents.descent) / 2))
    ctx.show_text(sample_text)
    display()
#end vert_align_font_metrics

vert_align_font_metrics()

## Characters And Glyphs

Text nowadays is almost universally encoded as [Unicode](http://unicode.org/). But there are different ways of representing Unicode text. In C, the `char` type is only 8 bits. Therefore, to keep Unicode text as an array of `char` values, the text is turned into [UTF-8](http://unicode.org/faq/utf_bom.html) encoding. Text files are also usually encoded this way. Cairo expects strings to be encoded the same way.

In Python 3, all in-memory text strings are Unicode-encoded. Each element of a `str` object corresponds to a Unicode *code point*. The closest equivalent of the C `char` array is the [`bytes`](https://docs.python.org/3/library/stdtypes.html#binary-sequence-types-bytes-bytearray-memoryview) type. In particular, it is possible to convert between Unicode strings and UTF-8-encoded bytes objects, e.g.

In [ ]:
s = "spell “Bézier” properly"
b = s.encode("utf-8")
s2 = b.decode("utf-8")
print(type(s), s)
print(type(b), b)
print(s == s2)

Qahirah lets you use Python strings wherever it makes sense to do so, and will automatically handle the translation to and from UTF-8 encoding for the underlying Cairo calls.

In a font, each character shape is called a *glyph*. In most font formats, each glyph is identified by a non-negative integer *glyph code*. Whereas Unicode is a standard character encoding, there is no such thing as a standard glyph encoding (beyond minimal conventions such as glyph 0 is always supposed to be the “missing” glyph, and glyph 1 is always supposed to be blank). Indeed, a font may offer alternative forms of the same character, ligatures etc.

The main Cairo functions for rendering and measuring Unicode-encoded text have alternatives that take arrays of glyph codes and placements. Cairo also offers the function [`cairo_scaled_font_text_to_glyphs`](http://cairographics.org/manual/cairo-cairo-scaled-font-t.html#cairo-scaled-font-text-to-glyphs) which will do a simpleminded conversion of Unicode text to glyphs representation—more elaborate functionality is left for layout libraries that understand something of OpenType font features and the like, such as [Pango](http://www.pango.org/). The rationale is that, having done whatever is necessary in the layout library to convert a run of text to a high-quality glyphs representation, including fine-tuning the relative positions of the glyphs, it can then be left to Cairo to do the actual low-level rendering or measuring of the text.

## Font Size Subtleties

The following cell demonstrates a subtle point about font rendering. A naïve scan-conversion of the font outline geometry can produce quite ugly results at all but the highest pixel resolutions. The fix for this is a technique called “hinting”, which actually *distorts* the font outlines in order to achieve more pleasing and consistent rendering results.

This example code exaggerates the effect of hinting: if you choose a scale factor less than 1, then the resolution at which the text is being rendered is deliberately lowered, before scaling up the results again (without antialiasing, to make the pixelation more apparent) for display. You can also turn the hinting on and off, and observe the results on the text. Note the following:

* With hinting off, the glyph shapes and positions remain the same regardless of scaling. This leads to noticeable irregularities in the rendering.
* With hinting on, the glyph shapes and positions undergo small adjustments to allow more regular rendering.

The effect is most noticeable at the smallest scaling values, and with antialiasing turned off.

In [ ]:
@interact \
  (
    fontspec = widgets.Text(value = "Serif"),
    scale = widgets.RadioButtons(options = list((str(x), x) for x in [0.5, 0.7, 1, 3, 10, 30]), value = 1),
    hint = widgets.Checkbox(value = True),
    antialias = widgets.Checkbox(value = True)
  )
def optical_size_demo(fontspec, scale, hint, antialias) :
    reset()
    sample_font = qah.FontFace.create_for_pattern(fontspec)
    temppix = qah.ImageSurface.create \
      (
        format = pix.format,
        dimensions = round(pix.dimensions * scale)
      )
    font_options = ctx.font_options
    font_options.hint_style = (CAIRO.HINT_STYLE_NONE, CAIRO.HINT_STYLE_DEFAULT)[hint]
    font_options.antialias = (CAIRO.ANTIALIAS_NONE, CAIRO.ANTIALIAS_DEFAULT)[antialias]
    (qah.Context.create(temppix)
        .set_source_colour(Colour.grey(1))
        .paint()
        .set_source_colour(text_colour)
        .set_matrix(Matrix.scale(scale))
        .translate(pix.dimensions * Vector(0.1, 0.5))
        .set_font_face(sample_font)
        .set_font_size(72)
        .set_font_options(font_options)
        .move_to((0, 0))
        .show_text(sample_text)
    )
    temppix.flush()
    (ctx
         .set_source
           (
             qah.Pattern.create_for_surface(temppix)
                .set_matrix(Matrix.scale(scale))
                .set_filter((CAIRO.FILTER_NEAREST, CAIRO.FILTER_GOOD)[scale > 1])
           )
        .paint()
    )
    display()
#end optical_size_demo

## The Font Matrix

Cairo does not just provide a call for setting the font size for drawing text: you can set a general **font matrix**, allowing for translation, scaling and rotation of the text. Think of `set_font_size` is a convenience wrapper for setting a matrix with just a scaling component. You will note that there is no `get_font_size` call in Cairo (which is why Qahirah does not provide a read/write `font_size` property); but you can get and set the entire `font_matrix`.

The following example shows some of the possibilities achievable with a general font matrix. Notice how the skew, in particular, gives a faux-italic effect: this is the actual technique used in Cairo (actually, in FreeType, which Cairo relies on for text rendering) to implement the `FT_SYNTHESIZE_OBLIQUE` option.

In [ ]:
@interact \
  (
    translate_x = (-40, +40, 10),
    translate_y = (-40, +40, 10),
    rotate = (-90, +90, 5),
    scale_x = (10, 50, 10),
    scale_y = (10, 50, 10),
    skew_x = (-1.1, +1.1, 0.1), # try to choose limits such that default value is 0
    skew_y = (-1.1, +1.1, 0.1),
  )
def font_matrix_demo(translate_x, translate_y, rotate, scale_x, scale_y, skew_x, skew_y) :
    reset()
    ctx.translate(pix.dimensions * Vector(0.1, 0.6))
    ctx.select_font_face("Sans-Serif", CAIRO.FONT_SLANT_NORMAL, CAIRO.FONT_WEIGHT_NORMAL)
    ctx.font_matrix = \
        (
            Matrix.translate((translate_x, translate_y))
        *
            Matrix.rotate(rotate * qah.deg)
        *
            Matrix.scale((scale_x, scale_y))
        *
            Matrix.skew((skew_x, skew_y))
        )
    ctx.move_to((0, 0))
    ctx.show_text(sample_text)
    ctx.show_text(sample_text)
    # do it twice to demonstrate where current point is left
    # after drawing each piece of text
    display()
#end font_matrix_demo

## Fitting Text To A Given Width

Sometimes you need to draw a sequence of lines containing variable amounts of text, within a column of fixed width. Some of the lines may be long: if you choose a font for all the text that is small enough not to truncate those lines, then the rest of the text may look too small. But if you choose a smaller font just for the long lines, then that looks odd, too.

A good compromise is to adjust the horizontal scaling component of the font matrix on the long lines, so that they will fit. (The vertical scaling is the same for all the lines.) This distorts the text, but I think this is not as bad as the other choices. Of course, the short lines don’t need to be distorted, so they can keep uniform scaling in their font matrix.

Anyway, see the effect for yourself:

In [ ]:
text_fit_widths = (25, 600, 25)
text_fit_sizes = (5, 30, 5)
text_fit_example_items = \
  (
    "ARABIC LIGATURE UIGHUR KIRGHIZ YEH WITH HAMZA ABOVE WITH ALEF MAKSURA ISOLATED FORM",
    "BALINESE MUSICAL SYMBOL COMBINING GONG",
    "CYRILLIC CAPITAL LETTER YU",
    "DEVANAGARI OM",
    "GREEK SMALL LETTER IOTA WITH DIALYTIKA AND TONOS",
    "MONGOLIAN LETTER SIBE DA",
    "TILDE",
  )

@interact \
  (
    fit_width = text_fit_widths,
    font_size = text_fit_sizes,
  )
def hor_text_fit_demo_1(fit_width, font_size) :
    reset()
    ctx.select_font_face("Sans-Serif", CAIRO.FONT_SLANT_NORMAL, CAIRO.FONT_WEIGHT_NORMAL)
    uniform_font_matrix = Matrix.scale(font_size)
    ctx.font_matrix = uniform_font_matrix
    line_spacing = ctx.scaled_font.font_extents.height
    ctx.move_to((20, 20))
    pos = ctx.current_point
    ctx.source_colour = line_colour
    (ctx
        .move_to((pos.x, 0))
        .rel_line_to((0, pix.dimensions.y))
        .move_to((pos.x + fit_width, 0))
        .rel_line_to((0, pix.dimensions.y))
        .set_line_width(1)
        .stroke()
    )
    ctx.source_colour = text_colour
    ctx.move_to(pos)
    for item in text_fit_example_items :
        pos = ctx.current_point
        ctx.font_matrix = uniform_font_matrix
        text_width = ctx.text_extents(item).advance.x
        if text_width > fit_width :
            ctx.font_matrix = Matrix.scale((fit_width / text_width, 1)) * uniform_font_matrix
        #end if
        ctx.show_text(item)
        ctx.move_to(pos + Vector(0, line_spacing))
    #end for
    display()
#end hor_text_fit_demo_1

You may notice that the above does not fit the text in the width exactly: it tends to be too wide or too narrow. I’m not sure why this is--some kind of rounding of text metrics?

An alternative, but more complex, way of doing it, is to render the text at uniform scaling into a separate `ImageSurface`, which is then non-uniformly scaled, with horizontal squashing as necessary, into the final image:

In [ ]:
@interact \
  (
    fit_width = text_fit_widths,
    font_size = text_fit_sizes,
  )
def hor_text_fit_demo_2(fit_width, font_size) :
    reset()
    #ctx.select_font_face("Sans-Serif", CAIRO.FONT_SLANT_NORMAL, CAIRO.FONT_WEIGHT_NORMAL)
    ctx.set_font_face(sample_font)
    uniform_font_matrix = Matrix.scale(font_size)
    ctx.font_matrix = uniform_font_matrix
    line_spacing = ctx.scaled_font.font_extents.height
    ctx.move_to((20, 20))
    pos = ctx.current_point
    ctx.source_colour = line_colour
    (ctx
        .move_to((pos.x, 0))
        .rel_line_to((0, pix.dimensions.y))
        .move_to((pos.x + fit_width, 0))
        .rel_line_to((0, pix.dimensions.y))
        .set_line_width(1)
        .stroke()
    )
    ctx.source_colour = text_colour
    ctx.move_to(pos)
    for item in text_fit_example_items :
        pos = ctx.current_point
        text_width = ctx.text_extents(item).advance.x
        if text_width > fit_width :
            text_bounds = ctx.text_extents(item).bounds
            offscreen = qah.ImageSurface.create \
              (
                format = CAIRO.FORMAT_ARGB32,
                dimensions = round(ctx.user_to_device_distance(text_bounds.dimensions))
              )
            (qah.Context.create(offscreen)
                .scale(offscreen.dimensions / text_bounds.dimensions) # match destination resolution
                .translate(- text_bounds.topleft)
                .set_operator(CAIRO.OPERATOR_SOURCE)
                .set_source_colour(Colour.grey(0, 0))
                .paint()
                .set_source_colour(ctx.source_colour)
                .set_scaled_font(ctx.scaled_font)
                .move_to((0, 0))
                .show_text(item)
            )
            offscreen.flush()
            (ctx
                .save()
                .set_source
                  (
                    qah.Pattern.create_for_surface(offscreen)
                        .set_extend(CAIRO.EXTEND_NONE)
                        .set_matrix
                          (
                                Matrix.scale(text_bounds.dimensions / offscreen.dimensions)
                            *
                                Matrix.translate(- text_bounds.topleft)
                            *
                                Matrix.scale(Vector(text_width / fit_width, 1))
                            *
                                Matrix.translate(- ctx.current_point)
                          )
                  )
                .paint()
                .restore()
            )
        else :
            ctx.show_text(item)
        #end if
        ctx.move_to(pos + Vector(0, line_spacing))
    #end for
    display()
#end hor_text_fit_demo_2